# Sequential pattern

In [1]:
from os import environ

environ.setdefault("HF_HOME", "/data/hf_models/")

'/data/hf_models/'

In [2]:
from a2a.types import AgentCapabilities, AgentCard, AgentSkill
from aap_core.agent import BaseAgent
from aap_core.chain import BaseLLMChain
from aap_core.orchestration import SequentialAgent
from aap_core.types import AgentMessage
from aap_transformers.chain import ChatCausalMultiTurnsChain
from pydantic import Field

/data/agent_design_pattern/src/transformers/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Define the agent

Create a simple agent for demonstration

In [3]:
class FunctionImplementationAgent(BaseAgent):
    chain: BaseLLMChain = Field(...)

    def execute(self, message: AgentMessage, **kwargs):
        self.state = "running"
        message = self.chain.invoke(message, **kwargs)
        self.state = "idle"
        message.execution_result = "success"
        message.origin = self.card.name
        return message

In [4]:
code_system_prompt = """You are a helpful coding assistant.
You task is to write a python function and return the implementation of the function.
Just the implementation, DO NOT comment anything else"""
code_user_prompt = "{query}"
code_chain = ChatCausalMultiTurnsChain(
    model="swiss-ai/Apertus-8B-Instruct-2509",
    system_prompt=code_system_prompt,
    user_prompt_template=code_user_prompt,
    device="cuda",
    max_new_tokens=8192)
code_chain.final_response_as_context("code")

doc_system_prompt = """You are a great developer avocate.
You task is to write a document and in-code explain for the function.
Just the comment and explanation, DO NOT change the logic of the provided function.
The response should contain function with docstring explanation. And DO NOT contain explanation outside of the code"""
doc_user_prompt = """Input query: {query}

Function implementation: {context_code}
"""
doc_chain = ChatCausalMultiTurnsChain(
    model="HuggingFaceTB/SmolLM3-3B",
    system_prompt=doc_system_prompt,
    user_prompt_template=doc_user_prompt,
    device="cuda",
    max_new_tokens=8192)
doc_chain.final_response_as_context("doc")

review_system_prompt = """You are a excellent code reviewer and refactor.
Given a function implementation and it explanation, your task is to review and code and correct if contains any mistake.
Some note:
- For the implementation, check if the orignal query and suggested implementation are match.
- Is there any syntax error in the code.
- For the explanation, verify if the docstring follows Google style docstring.
- In the docstring, make sure to have an example to call the function.

Make sure the final output only contain code, inline code comment and docstring, nothing else."""
review_user_prompt = """Input query: {query}

Function implementation: {context_doc}
"""
review_chain = ChatCausalMultiTurnsChain(
    model="ibm-granite/granite-4.0-h-1b",
    system_prompt=review_system_prompt,
    user_prompt_template=review_user_prompt,
    device="cuda",
    max_new_tokens=8192)

def state_callback(message: str):
    print(message)
code_skill = AgentSkill(
    id='code-skill',
    name="code skill",
    description="self-code skill",
    tags=['code']
)
code_card = AgentCard(
    name="code agent",
    description="self-code agent",
    skills=[code_skill],
    capabilities=AgentCapabilities(),
    default_input_modes=['text'],
    default_output_modes=['text'],
    url="localhost",
    version="0.1.0"
)
doc_skill = AgentSkill(
    id='doc-skill',
    name="doc skill",
    description="self-doc skill",
    tags=['doc']
)
doc_card = AgentCard(
    name="doc agent",
    description="self-doc agent",
    skills=[doc_skill],
    capabilities=AgentCapabilities(),
    default_input_modes=['text'],
    default_output_modes=['text'],
    url="localhost",
    version="0.1.0"
)
review_skill = AgentSkill(
    id='review-skill',
    name="review skill",
    description="self-review skill",
    tags=['review']
)
review_card = AgentCard(
    name="review agent",
    description="self-review agent",
    skills=[review_skill],
    capabilities=AgentCapabilities(),
    default_input_modes=['text'],
    default_output_modes=['text'],
    url="localhost",
    version="0.1.0"
)
sequential_skill = AgentSkill(
    id='sequential-skill',
    name="sequential skill",
    description="self-sequential skill",
    tags=['sequential']
)
sequential_card = AgentCard(
    name="sequential agent",
    description="self-sequential agent",
    skills=[sequential_skill],
    capabilities=AgentCapabilities(),
    default_input_modes=['text'],
    default_output_modes=['text'],
    url="localhost",
    version="0.1.0"
)
code_agent = FunctionImplementationAgent(chain=code_chain, card=code_card, state_change_callback=state_callback)
doc_agent = FunctionImplementationAgent(chain=doc_chain, card=doc_card, state_change_callback=state_callback)
review_agent = FunctionImplementationAgent(chain=review_chain, card=review_card, state_change_callback=state_callback)
agent = SequentialAgent(agents=[code_agent, doc_agent, review_agent], card=sequential_card, state_change_callback=state_callback)

CUDA-fused xIELU not available (No module named 'xielu') – falling back to a Python version.
For CUDA xIELU (experimental), `pip install git+https://github.com/nickjbrowning/XIELU`
Loading checkpoint shards: 100%|██████████| 2/2 [00:04<00:00,  2.24s/it]
The fast path is not available because one of `(selective_state_update, causal_conv1d_fn, causal_conv1d_update)` is None. Falling back to the naive implementation. To install follow https://github.com/state-spaces/mamba/#installation and https://github.com/Dao-AILab/causal-conv1d


In [5]:
query = "Write a python function to find the biggest but not exceed the given integer number. The found number must be a number in Fibonacci array."
message = agent.execute(AgentMessage(query=query), max_new_tokens=8192)

for response in message.responses:
    name, msg = response
    print(name)
    print(msg)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


sequential agent:running/((code agent:idle)-(doc agent:idle)-(review agent:idle))
sequential agent:running/((code agent:running)-(doc agent:idle)-(review agent:idle))
sequential agent:running/((code agent:idle)-(doc agent:idle)-(review agent:idle))
sequential agent:running/((code agent:idle)-(doc agent:running)-(review agent:idle))
sequential agent:running/((code agent:idle)-(doc agent:idle)-(review agent:idle))
sequential agent:running/((code agent:idle)-(doc agent:idle)-(review agent:running))
sequential agent:running/((code agent:idle)-(doc agent:idle)-(review agent:idle))
sequential agent:idle/((code agent:idle)-(doc agent:idle)-(review agent:idle))
review agent
Here is the reviewed and refactored code:

```python
def find_biggest_fibonacci(n):
    """
    Returns the largest Fibonacci number that does not exceed the given integer n.

    Args:
        n (int): The upper bound for the Fibonacci number.

    Returns:
        int: The largest Fibonacci number less than or equal to n.